In [1]:
X.groupby('BatchId')[["Amount", "Value", "Day","Hour","week_day", "weeks"]].agg(['count', 'min', 'max','mean'])

df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})

NameError: name 'X' is not defined